<p style="text-align: center;"><span style="color: #ff0000;"><strong><span style="font-size: x-large;">
    ANEXO 6: ALGORITMOS PKE</span></strong></span></p>

<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Realizado por:</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Gabriel Vacaro Goytia</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Ignacio Warleta Murcia</span></strong></span></p>

Este notebook contiene una implementación de los algoritmos fundamentales del esquema de cifrado post–cuántico Kyber, en particular el KyberPKE (Public Key Encryption). Kyber es un sistema criptográfico basado en el problema de Redes Lattice y es uno de los candidatos más destacados para ser parte del estándar de criptografía post-cuántica propuesto por el NIST.

Organizamos el anexo según el siguiente índice:

# Índice

1. [Introducción](#1.-Introducción)
2. [Configuración previa](#2.-Configuracion-previa)
3. [Algoritmo de generación de claves](#3.-Algoritmo-de-generacion-de-claves)
4. [Algoritmo de cifrado](#4.-Algoritmo-de-cifrado)
5. [Algoritmo de descifrado](#5.-Algoritmo-de-descifrado)
6. [Ejemplo de uso](#6.-Ejemplo-de-uso)

---
# 1. Introducción






Este notebook cuenta con los 3 algoritmos relativos al PKE de Kyber–KEM:

- Generación de claves (Key Generation) <br>
- Cifrado (Encryption) <br>
- Descifrado (Decryption) <br>

El propósito de esta implementación es comprender los detalles detrás de cada fase del esquema KyberPKE de manera didáctica, con especial énfasis en los aspectos técnicos que permiten asegurar la privacidad y la integridad de la comunicación en un entorno potencialmente afectado por computadoras cuánticas.


---
# 2. Configuracion previa

A continuación, se muestra la configuración previa a ejecutar y los parámetros a definir. En este caso hemos optado por un primo q relativamente pequeño, 743, dado su caracter didáctico, ya que, de esta manera se puede apreciar ligeramente el error que se produce al descifrar el mensaje. Por supuesto, el usuario es libre de cambiar este primo a su gusto y comprobar por su cuenta como se maneja el error.

In [1]:
#MODULOS A IMPORTAR
import numpy as np
import secrets

In [31]:
# Parámetros básicos
q = 743  # Un número primo pequeño típico en Kyber podría ser 3329
k = 3  # Tamaño del vector/matriz (varía según los estándares Kyber-512, 768, 1024)
mu_1 = 1.0  # Parámetro para la distribución de error más controlado (menor desviación estándar)

---
# 3. Algoritmo de generacion de claves

Primeramente, tenemos algunas funciones auxiliares que nos ayudan a generar una semilla aleatoria que utilizaremos para inicializar y generar la matriz pública A. Esta semilla garantiza que la generación de claves sea determinista y reproducible, pero a la vez, aleatoria.

In [3]:
#Funciones auxiliares 

# Función para generar una semilla aleatoria
def generate_seed(n=16):
    
    """
    Generación de semilla aleatoria de n bytes 
     
    Parámetros:
    - n: número de bytes (16 por defecto)
    
    Retorna:
    - Semilla aleatorio de n bytes.
    """
    return secrets.token_bytes(n)

def generate_matrix_A(seed, k, q):
    """
    Esta función genera una matriz aleatoria k × k con valores en el rango 
    [0,q), utilizando una semilla dada para asegurar la reproducibilidad de los números aleatorios.
     
    Parámetros:
    - seed: Un valor en bytes que se usa para inicializar la semilla del generador de números aleatorios.
    - k: Dimensión de la matriz cuadrada que se va a generar.
    - q: módulo sobre el que se trabaja.
    
    Retorna:
    - Matriz aleatorio kxk con valores en [0,q)
    """
    np.random.seed(int.from_bytes(seed, "big") % (2**32)) # Convertir semilla a entero usando big-endian limitado a 32 bits
    return np.random.randint(0, q, size=(k, k)) 

def sample_error(mu, k, q, seed=None):
    """
    Esta función genera un vector de errores siguiendo una distribución normal centrada en 0 
    con desviación estándar mu en módulo q. Si no se introduce semilla, se utilizará 
    el estado actual del generador aleatorio de NumPy, permitiendo que los valores 
    generados varíen en cada ejecución. 
     
    Parámetros:
    - mu: Desviación estándar de la distribución normal.
    - k: Número de filas del vector (dimensión del error).
    - q: Módulo sobre el cual se trabaja.
    - seed: Valor opcional para fijar la semilla del generador aleatorio.
    
    Retorna:
    - Vector k dimensional de valores aleatorios en el rango [0,q).
    """
    if seed is not None:
        state = np.random.get_state()  # Guarda el estado actual del generador
        if isinstance(seed, bytes):  # Si es de tipo Bytes:
            seed = int.from_bytes(seed, "big") % (2**32) # Refundicion a entero en rango de 32 bits.
        np.random.seed(seed)  # Fija la semilla para reproducibilidad

    error = np.round(np.random.normal(0, mu, size=(k, 1))).astype(int) % q  # Genera el vector de errores

    if seed is not None:
        np.random.set_state(state)  # Restaura el estado original del generador

    return error
    

A continuacion se explica paso a paso el algoritmo:

1. **Generación de la matriz pública A:**
   En el segundo paso se define A como una matriz pública de tamaño $k∙k$ cuyos elementos son tomados de un conjunto de enteros módulo $q$ (del conjunto $R_q$). PSR es un proceso de muestreo que genera valores aleatorios para A.

2. **Generación de los vectores secreto y de error:**
   En el tercer y cuarto pasos se generan los vectores $s$ y $e$. El vector secreto $s$ es un vector aleatorio de $k$ elementos, cuyos valores provienen de una distribución  $\beta_{\mu_1}$. Esta distribución controla la cantidad de ruido o error en el sistema. De manera similar, el vector $e$ también se genera a partir de la misma distribución. El vector $e$ representa el error que se agrega a la multiplicación de $A$ y $s$ para asegurar la seguridad del sistema.

4. **Cálculo de la clave pública:**
   Seguidamente, se calcula la clave pública $b$ como el resultado de multiplicar la matriz $A$ por el vector secreto $s$ y luego agregar el vector de error $e$. Esto crea un error en el producto para hacer que sea difícil de resolver para un atacante, incluso si conoce la matriz $A$.

5. **Construcción de las claves pública y privada:**
   En los pasos sexto y séptimo se construyen las claves pública y privada. En primer lugar, la clave pública $pk$ se forma concatenando el vector $b$ con la semilla $seed A$. Esto asegura que cualquier persona que conozca $pk$ pueda verificar la autenticidad de $b$ y generar su propia clave compartida, pero no pueda conseguir fácilmente $s$. Por otro lado, la clave privada $sk$ es simplemente el vector secreto $s$.

6. **Devolución de las claves:**
   Por último, el algoritmo devuelve la clave pública $pk$ y la clave privada $sk$.


In [4]:
# Algoritmo 1: Generación de Claves G'
def key_generation():
    """
    Generación de claves pública y secreta para un esquema basado en LWE; relativo al PKE.
    
    Retorna:
    - Clave pública (b, seed_A): Vector b y semilla para regenerar la matriz A.
    - Clave secreta s: Vector secreto utilizado para descifrar.
    """
    seed_A = generate_seed() # Se genera una semilla aleatoria para construir la matriz A de forma determinista
    A = generate_matrix_A(seed_A, k, q) # Se genera la matriz A de dimensión k x k en módulo q
    s = sample_error(mu_1, k, q)  # Se genera el vector secreto s con distribución gaussiana
    e = sample_error(mu_1, k, q)  # Se genera el vector error la misma distribución
    b = (A @ s + e) % q   # Se calcula el vector b como b = (A * s + e) mod q
    
    return (b, seed_A), s

---
# 4. Algoritmo de cifrado

El algoritmo de cifrado recibe como parámetros de entrada la clave pública $pk$ generada en el paso anterior, y con ella el vector $b$ y la $seed A$ que usará para la generacion de la matriz $A$ tranpuesta. Posteriormente sigue los siguientes pasos:

1. **Generación del vector aleatorio $r$:**  
   Seguidamente, generamos un vector $r$ aleatorio de longitud $k$, cuyos valores se extraen de una distribución $\beta_{\mu_1}$ para generar ruido en el cifrado.

2. **Generación de los vectores de error $e_1$ y $e_2$:**  
   En los pasos cuarto y quinto, generamos los vectores de error $e_1$ y $e_2$.  
   - El vector $e_1$ es de longitud $k$ y se genera de acuerdo con la distribución $\beta_{\mu_2}$, que controla el error en la multiplicación de matrices durante el proceso de cifrado.  
   - El vector $e_2$ es un vector de longitud $1$, generado también a partir de la misma distribución. Este se utilizará en el cálculo del componente de la clave en el mensaje cifrado.

3. **Cálculo de $u$ y $v$:**  
   En los siguientes pasos, calculamos los componentes del mensaje cifrado:  
   - El primer componente, $u$, se calcula multiplicando la matriz $A^T$ por el vector $r$ y luego sumando el vector de error $e_1$. Esto asegura que $u$ dependa de la clave pública y del ruido.  
   - El segundo componente, $v$, se calcula tomando el producto $b$ (la clave pública) con el vector $r$, sumando el error $e_2$ y finalmente añadiendo el mensaje $m$. Esto garantiza que $v$ contenga la información del mensaje cifrado de manera que solo la persona con la clave privada pueda descifrarlo.

4. **Generación del mensaje cifrado final:**  
   Por último, concatenamos $u$ y $v$ como el mensaje cifrado final $c$, el cual devuelve el algoritmo.


In [5]:
#Algoritmo de Cifrado

def encrypt(pk, m, seed=None):
    """
    Esta función implementa el algoritmo de cifrado del PKE modificado con el uso opcional 
    de una semilla para garantizar la reproducibilidad de los valores aleatorios 
    generados.
    
    Parámetros:
    - pk: Clave pública (b, seed_A), donde:
        - b: Vector b generado en la fase de generación de claves.
        - seed_A: Semilla usada para reconstruir la matriz A.
    - m: Mensaje a cifrar (normalmente representado como un número en ℤ_q).
    - seed: Valor opcional que, si se proporciona, permite generar errores y aleatoriedad 
      de forma determinista mediante el uso de una función hash.

    Retorna:
    - (u, v): Par de valores cifrados.
    - r: Vector de aleatorización utilizado en el cifrado.
    - e_1: Vector de error agregado a u.
    - e_2: Valor de error agregado a v.
    """
    b, seed_A = pk # valores de la clave pública
    if seed:     # Si se proporciona una semilla, se usa SHA3-256 para derivar valores deterministas
        hash_output = hashlib.sha3_256(seed).digest() # Hash de la semilla
        r_seed = hash_output[:4]  # Usar solo 4 bytes (32 bits)
        e1_seed = hash_output[4:8]
        e2_seed = hash_output[8:12]
        # Se generan los vectores de aleatorización y errores de forma determinista
        r = sample_error(mu_1, k, q, int.from_bytes(r_seed, 'big'))
        e_1 = sample_error(mu_1, k, q, int.from_bytes(e1_seed, 'big'))
        e_2 = sample_error(mu_1, k, q, int.from_bytes(e2_seed, 'big'))
    else:
        # Si no hay semilla no será reproducible
        r = sample_error(mu_1, k, q)
        e_1 = sample_error(mu_1, k, q)
        e_2 = sample_error(mu_1, k, q)
    
    A = generate_matrix_A(seed_A, k, q)     
    u = (A.T @ r + e_1) % q  # Se calcula u = (Aᵀ * r + e₁) mod q
    v = (b.T @ r + e_2 + m) % q # Se calcula v = (bᵀ * r + e₂ + m) mod q
    return (u, v), r, e_1, e_2

---
# 5. Algoritmo de descifrado

El propósito del algoritmo de descifrado es recuperar el mensaje original a partir del mensaje cifrado $c$ y la clave secreta $sk$.  
- La clave secreta $sk$ es el vector secreto $s$, que el receptor mantiene de forma privada.  
- El mensaje cifrado $c$, como vimos en el esquema anterior, es el resultado del algoritmo de cifrado y consiste en la concatenación de los componentes $u$ y $v$.

### Pasos del descifrado  

1. **Descomposición del mensaje cifrado:**  
   Primero, se descompone $c$ para obtener $u$ y $v$, los cuales son necesarios para recuperar $m$.  

2. **Recuperación del mensaje original:**  
   Aplicando la ecuación correspondiente y usando la clave secreta $sk$, se obtiene el mensaje original $m$.  


In [6]:
# Algoritmo 3: Descifrado D(sk, c)
def decrypt(sk, c):
    u, v = c
    s = sk  # La clave secreta s

    # Calcular m = v - s^T * u
    m_recovered = (v - (s.T @ u)) % q

    return m_recovered

def decrypt(sk, c):
    """
    implementa el algoritmo de descifrado basado en LWE; relativo al PKE, recuperando el mensaje original 
    a partir del par cifrado (u, v) y la clave secreta s.

    Parámetros:
    - sk: Clave secreta utilizada en la generación de claves.
    - c: Texto cifrado (u, v), donde:
      - u: Vector generado durante el cifrado.
      - v: Valor resultante que contiene el mensaje más ruido.

    Retorna:
    - El mensaje descifrado en el dominio ℤ_q.
    """
    u, v = c  
    s = sk  
    m_recovered = (v - (s.T @ u)) % q # Se recupera el mensaje m utilizando la relación v - sᵀ * u mod q
    
    return m_recovered

---
# 6. Ejemplo de uso

A continuación se muestra un ejemplo de uso concatenando los tres algoritmos:

In [32]:
import numpy as np

def print_header(title):
    print("\n" + "═" * 50)
    print(f"{title.center(50)}")
    print("═" * 50 + "\n")

def print_section(title, content):
    print_header(title)
    print(content, "\n")

# 1. Generar las claves
print_header("🔑 GENERACIÓN DE CLAVES 🔑")
public_key, secret_key = key_generation()

print("📜 Clave Pública (pk):")
print(f"🟢 b: {public_key[0]}")
print(f"🟢 seed_A: {public_key[1]}\n")

print("🔐 Clave Secreta (sk):")
print(secret_key, "\n")

# 2. Definir y mostrar el mensaje original
print_header("📩 MENSAJE ORIGINAL 📩")
m = np.array([1, 2, 3]).reshape((k, 1))  # Mensaje original (vector columna)
print(m, "\n")

# 3. Cifrado del mensaje
print_header("🔒 CIFRADO DEL MENSAJE 🔒")
ciphertext, r, e_1, e_2 = encrypt(public_key, m)

print("🔑 Cifrado (u, v):")
print(f"🟣 u: {ciphertext[0]}")
print(f"🟣 v: {ciphertext[1]}\n")

print("📊 Vectores y errores generados durante el cifrado:")
print(f"🟡 r (vector aleatorio r): {r}")
print(f"🟡 e_1 (error en u): {e_1}")
print(f"🟡 e_2 (error en v): {e_2}\n")

# 4. Descifrar el mensaje
print_header("🔓 DESCIFRADO DEL MENSAJE 🔓")
recovered_message = decrypt(secret_key, ciphertext)

print("📩 Mensaje recuperado:")
print(recovered_message, "\n")

# 5. Comparación entre mensaje original y recuperado
print_header("📊 COMPARACIÓN ENTRE MENSAJE ORIGINAL Y RECUPERADO 📊")
comparison = np.array_equal(m, recovered_message)
status = "✅ Coincide" if comparison else "❌ No coincide"
print(f"Resultado de la comparación: {status}\n")


══════════════════════════════════════════════════
             🔑 GENERACIÓN DE CLAVES 🔑             
══════════════════════════════════════════════════

📜 Clave Pública (pk):
🟢 b: [[105]
 [581]
 [700]]
🟢 seed_A: b'`\xd2\x92\x835P7\x85\xca\x0c\xa3Z,\x97\xe93'

🔐 Clave Secreta (sk):
[[742]
 [  0]
 [  0]] 


══════════════════════════════════════════════════
               📩 MENSAJE ORIGINAL 📩               
══════════════════════════════════════════════════

[[1]
 [2]
 [3]] 


══════════════════════════════════════════════════
             🔒 CIFRADO DEL MENSAJE 🔒              
══════════════════════════════════════════════════

🔑 Cifrado (u, v):
🟣 u: [[519]
 [320]
 [331]]
🟣 v: [[225]
 [226]
 [226]]

📊 Vectores y errores generados durante el cifrado:
🟡 r (vector aleatorio r): [[  1]
 [742]
 [  1]]
🟡 e_1 (error en u): [[  1]
 [  1]
 [742]]
🟡 e_2 (error en v): [[  0]
 [  0]
 [742]]


══════════════════════════════════════════════════
            🔓 DESCIFRADO DEL MENSAJE 🔓            
════

---
A continuación, podemos cambiar los parámetros para un mejor descifrado del mensaje.

Por ejemplo, una opción puede ser:
- q = 109
- k = 3
- mu_1 = 0.5

In [37]:
# Parámetros básicos
q = 109  # Un número primo pequeño típico en Kyber podría ser 3329
k = 3  # Tamaño del vector/matriz (varía según los estándares Kyber-512, 768, 1024)
mu_1 = 0.5  # Parámetro para la distribución de error más controlado (menor desviación estándar)

In [38]:
import numpy as np

def print_header(title):
    print("\n" + "═" * 50)
    print(f"{title.center(50)}")
    print("═" * 50 + "\n")

def print_section(title, content):
    print_header(title)
    print(content, "\n")

# 1. Generar las claves
print_header("🔑 GENERACIÓN DE CLAVES 🔑")
public_key, secret_key = key_generation()

print("📜 Clave Pública (pk):")
print(f"🟢 b: {public_key[0]}")
print(f"🟢 seed_A: {public_key[1]}\n")

print("🔐 Clave Secreta (sk):")
print(secret_key, "\n")

# 2. Definir y mostrar el mensaje original
print_header("📩 MENSAJE ORIGINAL 📩")
m = np.array([1, 2, 3]).reshape((k, 1))  # Mensaje original (vector columna)
print(m, "\n")

# 3. Cifrado del mensaje
print_header("🔒 CIFRADO DEL MENSAJE 🔒")
ciphertext, r, e_1, e_2 = encrypt(public_key, m)

print("🔑 Cifrado (u, v):")
print(f"🟣 u: {ciphertext[0]}")
print(f"🟣 v: {ciphertext[1]}\n")

print("📊 Vectores y errores generados durante el cifrado:")
print(f"🟡 r (vector aleatorio r): {r}")
print(f"🟡 e_1 (error en u): {e_1}")
print(f"🟡 e_2 (error en v): {e_2}\n")

# 4. Descifrar el mensaje
print_header("🔓 DESCIFRADO DEL MENSAJE 🔓")
recovered_message = decrypt(secret_key, ciphertext)

print("📩 Mensaje recuperado:")
print(recovered_message, "\n")

# 5. Comparación entre mensaje original y recuperado
print_header("📊 COMPARACIÓN ENTRE MENSAJE ORIGINAL Y RECUPERADO 📊")
comparison = np.array_equal(m, recovered_message)
status = "✅ Coincide" if comparison else "❌ No coincide"
print(f"Resultado de la comparación: {status}\n")


══════════════════════════════════════════════════
             🔑 GENERACIÓN DE CLAVES 🔑             
══════════════════════════════════════════════════

📜 Clave Pública (pk):
🟢 b: [[  0]
 [108]
 [  0]]
🟢 seed_A: b'\xe2\xd3\xabPX\xb4\x14\xd0E\xa7\x11\xca"\xc4\xee\xe1'

🔐 Clave Secreta (sk):
[[0]
 [0]
 [0]] 


══════════════════════════════════════════════════
               📩 MENSAJE ORIGINAL 📩               
══════════════════════════════════════════════════

[[1]
 [2]
 [3]] 


══════════════════════════════════════════════════
             🔒 CIFRADO DEL MENSAJE 🔒              
══════════════════════════════════════════════════

🔑 Cifrado (u, v):
🟣 u: [[ 0]
 [19]
 [41]]
🟣 v: [[1]
 [2]
 [3]]

📊 Vectores y errores generados durante el cifrado:
🟡 r (vector aleatorio r): [[0]
 [0]
 [1]]
🟡 e_1 (error en u): [[108]
 [  1]
 [  0]]
🟡 e_2 (error en v): [[0]
 [0]
 [0]]


══════════════════════════════════════════════════
            🔓 DESCIFRADO DEL MENSAJE 🔓            
══════════════════════